### 1.加载当前周期的干扰类型为“外部干扰”的小区,按经纬度排序
### 2.for循环，将本小区与目标小区关联起来（目标小区经\纬度小于0.01）
### 3.计算本小区与目标小区距离，如果小于则保存
### 4.找出汇聚区域(本小区的周边1km小区及目标小区周边1km小区作为汇聚区域)

### 5.对汇聚区域进行编号

In [15]:
import folium

import pandas as pd
import numpy as np
#sns.set()
## 第一部分
df = pd.read_csv('干扰小区4g.csv')
#df.columns = ['Cell','Lon','Lat','ClusterId','Id']
df= df[['y','x','j']]

Celllist = df['j']
Celllist = Celllist.drop_duplicates()
Celllist = np.array(Celllist)

m = folium.Map(location=[30.576865, 114.261929], zoom_start=5)
point_layer = folium.FeatureGroup(name="Query Search")

colorlist = ['Pink','Crimson','PaleVioletRed','HotPink','DeepPink','MediumVioletRed','Orchid','Thistle','Plum','Violet','Magenta','Fuchsia','DarkMagenta','Purple','MediumOrchid','DarkViolet','DarkOrchid','Indigo','BlueViolet','MediumPurple','MediumSlateBlue','SlateBlue','DarkSlateBlue','Lavender','GhostWhite','Blue','MediumBlue','MidnightBlue','DarkBlue','Navy','RoyalBlue','CornflowerBlue','LightSteelBlue','LightSlateGray','SlateGray','DodgerBlue','AliceBlue','SteelBlue','LightSkyBlue','SkyBlue','DeepSkyBlue','LightBlue','PowderBlue','CadetBlue','Azure','LightCyan','PaleTurquoise','Cyan','Aqua','DarkTurquoise','DarkSlateGray','DarkCyan','Teal','MediumTurquoise','LightSeaGreen','Turquoise','Aquamarine','MediumAquamarine','MediumSpringGreen','MintCream','SpringGreen','MediumSeaGreen','SeaGreen','Honeydew','LightGreen','PaleGreen','DarkSeaGreen','LimeGreen','Lime','ForestGreen','Green','DarkGreen','Chartreuse','LawnGreen','GreenYellow','DarkOliveGreen','YellowGreen','OliveDrab','Beige','LightGoldenrodYellow','Ivory','LightYellow','Yellow','Olive','DarkKhaki','LemonChiffon','PaleGoldenrod','Khaki','Gold','Cornsilk','Goldenrod','DarkGoldenrod','FloralWhite','OldLace','Wheat','Moccasin','Orange','PapayaWhip','BlanchedAlmond','NavajoWhite','AntiqueWhite','Tan','BurlyWood','Bisque','DarkOrange','Linen','Peru','PeachPuff','SandyBrown','Chocolate','SaddleBrown'
             ,'Seashell','Sienna','LightSalmon','Coral','OrangeRed','DarkSalmon','Tomato','MistyRose','Salmon','LightCoral']

for i,j in zip(Celllist,colorlist):
    #print(i,j)
    df1 = df.loc[(df['j'] == i)]
    df1 = np.array(df1)
    #print(c)
    for lat,lng,nameP in df1:
        point_layer.add_child(folium.CircleMarker(location=[lat, lng], radius=2,
            popup=str(nameP) + " Lat: " + str(lat) + " , Lon: " + str(lng), 
            tooltip=str(nameP) + " Lat: " + str(lat) + " , Lon: " + str(lng),
            fill=True,  # Set fill to True
            color = j,
            fill_opacity=1.0)).add_to(m)

m.add_child(point_layer)
m.add_child(folium.LayerControl())
m.save("现场干扰小区汇聚.html")
#m

In [1]:
import pandas as pd
import numpy as np

#1、加载外部干扰小区
df = pd.read_csv('干扰小区4g.csv')
#df = df.sort_values(by=["lon","lat"],ascending=True)  # by指定按哪列排序。ascending表示是否升序

CellList = list(df['Cell'])
for  i in CellList:
    sourcedata = df.loc[df['Cell']== i]
    sourcedata = sourcedata.reset_index(drop=True)

    #过滤掉经纬度相差大于0.01的小区，经度1分 = 1.42km  ,纬度1分 = 大约1.85km
    df1 = df.loc[  ( abs(df['lon'] - float(sourcedata['lon']) ) < 0.01  ) | ( abs(df['lat'] -  float(sourcedata['lat']) ) < 0.01 )]    
    df1 = df1.reset_index(drop=True)

    
    for j in range(len(df1)):
        targetdata = pd.DataFrame(df1.iloc[j])
        targetdata = targetdata.T
        targetdata.columns =['Cell2','lon2','lat2']

        
        targetdata1 = targetdata.reset_index(drop=True)
        newdata = pd.concat( [sourcedata,targetdata1],axis=1 )
        
        
        from math import radians, cos, sin, asin, sqrt
        #公式计算两点间距离（m）
        def geodistance(lng1,lat1,lng2,lat2):
            lng1, lat1, lng2, lat2 = map(radians, [float(lng1), float(lat1), float(lng2), float(lat2)]) # 经纬度转换成弧度
            dlon=lng2-lng1
            dlat=lat2-lat1
            a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
            distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km
            distance=round(distance/1000,3)
            return distance

        lng1 = newdata['lon'].astype(float)
        lat1 = newdata['lat'].astype(float)
        lng2 = newdata['lon2'].astype(float)
        lat2 = newdata['lat2'].astype(float)

        dis = geodistance( lng1,lat1,lng2,lat2 )

        newdata['dis'] =  dis

        if float(newdata['dis']) < 1:
            newdata.to_csv('CgiCluster4g.csv', index=False ,mode='a', header=False )
            
        else:
            continue

KeyError: 'Cell'